In [2]:
import json
import requests
from datetime import datetime
from bs4 import BeautifulSoup
'''
1/14/24 - Currently retrieves 'title' from googlebooksAPI, converts HTML response into a string and writes it to a text file.
        googlebooks API returns the whole first page of search results for any given search (only 10 results); need to see if
        there's a way to expand the search results for manga that have many volumes; 
        EX: Say you search for undead unluck, 16 total volumes but, search page only shows 10 results. We may not get 
        the newest volume we're looking to keep track of if we just search "Undead Unluck" rather than "Undead Unluck Vol. 15"
        
            New Variables: Title - title of manga we're trying to search for
                            Response - Dict that contains the results of our query 
        
1/16/24 - Adjusted Google Books API call to include maxResults parameter, which allows us to return a greater number of results.

1/17/24 - Played around with the 'Response' Variable to figure out the best way to retrieve the data we need; 
            Created testDict variable to test data manipulation and it currently contains title: publishedDate
            New Variables: temp - temporarily stores Tuple returned from Response.items() so we can retrieve 'items' data
                            Volumes - Copies List of 'items' data from temp variable
                            testDict - Dict that contains the returned titles as keys and the published dates as values
                            
1/18/24 - Tested with datetime library and was able to convert publishedDate string to a date object to be used for 
            comparison to today's date;
            Updated maxResults to return the max number of results, 40; 
            *** Created new python file upcomingMangaReleases.py and turned the search into a function, upcomingReleases(Title)
                that returns a Dictionary of unreleased volumes
                - Created function called sendReleaseDate() that emails an appointment on the release date of the upcoming 
                    releases
                
                (based on code from https://rmsol.de/2018/06/17/Emails-and-Appointments-with-Outlook-and-Python/)
            New Variables: pDate - converts publishedDate string to a comparable Date object
                            

TO DO:  Look into filtering out volumes that have already come out; may need to look into python system libraries to 
        obtain date for comparison, as well as convert publishedDate to a comparable time format.
'''

Title="Undead Unluck, Vol"
#Title="Spy x Family, Vol"


# Creats a Dict object with the results of our query
Response = requests.get(f"https://www.googleapis.com/books/v1/volumes?q={Title}&maxResults=40").json()

# Converts reponse from API to a list so we can remove the 'kind' and 'totalItems' data; 
# Search result data is stored in tuple 'Volumes'

temp = list(Response.items())       # Converts Tuple into a List for easier traversal

Volumes = temp[2][1]               # Copies List of 'items' data from temp variable

SearchData = {}                    # Initialize dictionary for Title:PublishedDate
SortedResults = {}                  # Initialize Dictionary for unreleased Title:PublishedDate

i = 0

# Populates SearchData
for entry in Volumes:
    SearchData[Volumes[i].get('volumeInfo').get('title')] = Volumes[i].get('volumeInfo').get('publishedDate')
    i=i+1
    
    
# traverses dictionary of titles and published dates and only prints entries with a valid PublishedDate that is 
# sometime in the future and that contain the Title
for key in SearchData.keys():
    
    # Confirms that publishedDate has data as well as a full length date
    if SearchData[key] != None and len(SearchData[key]) == 10:
        pDate = datetime.strptime(SearchData[key], '%Y-%m-%d').date()   
    
    # Only prints out titles that haven't come out yet
    if SearchData[key] != None and Title in str(key) and datetime.now().date() < pDate:
        print(f"Title: {key} Published Date: {SearchData[key]}")
        SortedResults[key] = pDate
        #pass
        
'''
#Testing loop that prints out all the titles returned from our query
i = 0
for x in Volumes:
    VolumeTitle = Volumes[i].get('volumeInfo').get('title')
    PublishedDate = Volumes[i].get('volumeInfo').get('publishedDate')
    print(f"Volume Title: {VolumeTitle} Release Date: {PublishedDate}")
    i=i+1


#Testing out how to effectively retrieve the data we need from the Volumes list
print(type(Volumes))
print(type(Volumes[0]))
print(type(Volumes[0].get('volumeInfo')))
print(Volumes[0].get('volumeInfo').get('title'))



#soup = BeautifulSoup(response.text, "html.parser")


#Creates query.txt in working directory and writes html response to file for analysis. 
with open("query.txt", "w") as file:
    file.write(str(soup))
'''

Title: Undead Unluck, Vol. 15 Published Date: 2024-04-16
Title: Undead Unluck, Vol. 16 Published Date: 2024-07-16


'\n#Testing loop that prints out all the titles returned from our query\ni = 0\nfor x in Volumes:\n    VolumeTitle = Volumes[i].get(\'volumeInfo\').get(\'title\')\n    PublishedDate = Volumes[i].get(\'volumeInfo\').get(\'publishedDate\')\n    print(f"Volume Title: {VolumeTitle} Release Date: {PublishedDate}")\n    i=i+1\n\n\n#Testing out how to effectively retrieve the data we need from the Volumes list\nprint(type(Volumes))\nprint(type(Volumes[0]))\nprint(type(Volumes[0].get(\'volumeInfo\')))\nprint(Volumes[0].get(\'volumeInfo\').get(\'title\'))\n\n\n\n#soup = BeautifulSoup(response.text, "html.parser")\n\n\n#Creates query.txt in working directory and writes html response to file for analysis. \nwith open("query.txt", "w") as file:\n    file.write(str(soup))\n'